# Module 1 Hands-On: Stock & Company News Analyzer

## Overview

In this exercise, you'll build a **financial analysis agent** that demonstrates the SDK's "code generation for non-coding" philosophy. The agent will:

- Write Python code to fetch stock data
- Execute the code using Bash
- Analyze the results and generate insights

**Key Insight**: You're building a *finance agent*, not a coding assistant. The agent uses code as a tool to accomplish financial analysis tasks.

## Learning Objectives

By the end of this exercise, you'll understand:

1. **`query()` vs `ClaudeSDKClient`**: When to use each approach
2. **Tool Configuration**: Using `Write`, `Bash`, and `Read` tools
3. **Message Streaming**: Processing agent responses in real-time
4. **Message Types**: Understanding different message types
5. **Permission Modes**: Controlling agent autonomy

## Prerequisites

### Environment Setup

This module has its own self-contained uv environment. To set up:

```bash
# Navigate to this directory
cd notebooks/exercises/module_1

# Install dependencies with uv
uv sync

# Create .env file (if not using parent project's .env)
cp .env.example .env
# Edit .env and add your ANTHROPIC_API_KEY

# Start Jupyter from this directory
uv run jupyter notebook
```

**Important**: The exercises are designed to run from the `module_1` directory. All generated files will be saved to the `tmp/` subdirectory.

## Setup

First, let's configure the notebook environment:

In [13]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
from pathlib import Path

# Try to load .env from current directory first, then parent project
env_file = Path(".env")
if not env_file.exists():
    # Fall back to parent project's .env
    env_file = Path("../../../.env")

if env_file.exists():
    load_dotenv(env_file)
    print(f"✓ Loaded environment from {env_file}")
else:
    print("⚠️  No .env file found. Please create one with ANTHROPIC_API_KEY")

print("✓ Notebook environment configured")

✓ Loaded environment from ../../../.env
✓ Notebook environment configured


In [14]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

✓ API key found (length: 108 characters)


## Helper Functions

Let's create utilities to make message output more readable:

In [15]:
import json
from claude_agent_sdk import (
    AssistantMessage, UserMessage, SystemMessage, ResultMessage,
    TextBlock, ToolUseBlock, ToolResultBlock
)

def print_message(message):
    """Pretty print agent messages with appropriate formatting."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        # Print session info from init messages
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"\nSession initialized: {session_id[:8]}...")
    
    elif msg_type == "AssistantMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"\nAssistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"\nTool: {block.name}")
                    if hasattr(block, 'input'):
                        # Show description first if available
                        if 'description' in block.input:
                            print(f"   → {block.input['description']}")
                        # Show key arguments
                        if block.name == "Write" and 'file_path' in block.input:
                            print(f"   File: {block.input['file_path']}")
                        elif block.name == "Bash" and 'command' in block.input:
                            cmd = block.input['command']
                            if len(cmd) > 100:
                                cmd = cmd[:100] + "..."
                            print(f"   Command: {cmd}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"\nTool Error: {block.content}")
                    else:
                        content = str(block.content)
                        # Truncate long outputs
                        if len(content) > 500:
                            content = content[:500] + "...\n(output truncated)"
                        print(f"\nTool Result:\n{content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\nCost: ${message.total_cost_usd:.4f} | Time: {message.duration_ms/1000:.1f}s")
            print("=" * 60)

print("✓ Helper functions loaded")

✓ Helper functions loaded


---

## Part 1: Using `query()` for Single-Shot Analysis

### When to Use `query()`

`query()` is perfect for:
- One-off tasks
- Independent operations
- Simple automation
- When you don't need conversation history

### Exercise 1A: Basic Stock Analysis

Let's start with a simple task: analyze a single stock's recent performance.

In [16]:
from claude_agent_sdk import query, ClaudeAgentOptions
from pathlib import Path
import os

async def basic_stock_analysis():
    """Use query() for a single-shot stock analysis task."""
    
    print("=" * 60)
    print("EXERCISE 1A: Basic Stock Analysis with query()")
    print("=" * 60)
    
    # Get current working directory (should be module_1)
    cwd = Path.cwd()
    
    # Ensure tmp directory exists
    tmp_dir = cwd / "tmp"
    tmp_dir.mkdir(exist_ok=True)
    
    # Get absolute path to tmp directory
    tmp_path = str(tmp_dir.absolute())
    
    print(f"Working directory: {cwd}")
    print(f"Temp directory: {tmp_path}")
    
    # Configure the agent
    options = ClaudeAgentOptions(
        # Allow code generation and execution tools
        allowed_tools=["Write", "Bash", "Read"],
        
        # Auto-approve tool usage for smooth demo
        # In production, use permission_mode="default" with approval callbacks
        permission_mode="bypassPermissions",
        
        # Use Claude Sonnet for cost-effectiveness
        model="claude-sonnet-4-5",
        
        # Limit turns to prevent runaway execution
        max_turns=10,
        
        # Set working directory to current directory (module_1)
        cwd=str(cwd),
        
        # Custom system prompt to guide agent
        system_prompt=f"""You are a financial analysis assistant. You have access to Write, Bash, and Read tools.

IMPORTANT INSTRUCTIONS:
1. **Package Management**: This project uses uv for Python package management
   - To run Python scripts, use: `uv run python script.py`
   - All required packages (yfinance, pandas, matplotlib, etc.) are pre-installed
   - DO NOT use pip install - packages are already available via uv

2. **File Storage**: Save ALL generated files to this specific directory: {tmp_path}
   - Use ABSOLUTE PATHS when writing files
   - Example: Write scripts to `{tmp_path}/analysis.py`
   - Example: Save data to `{tmp_path}/stock_data.csv`
   - Example: Save results to `{tmp_path}/analysis_results.json`
   - DO NOT use /tmp or any other temp directory

3. **Workflow**: 
   - Write Python scripts with full path: `{tmp_path}/script_name.py`
   - Execute with `uv run python {tmp_path}/script_name.py`
   - Read results from `{tmp_path}/result_file.json`

Write clean, well-commented Python code for financial analysis tasks."""
    )
    
    # Define the task
    prompt = f"""
    Analyze Apple (AAPL) stock performance over the last 3 months.
    
    Write Python code using yfinance to:
    1. Fetch the stock data
    2. Calculate basic statistics (mean price, volatility, return)
    3. Save the results to a JSON file
    
    Remember: Use the absolute path {tmp_path} for all files.
    
    Then analyze the results and provide insights.
    """
    
    # Execute the task
    async for message in query(prompt=prompt, options=options):
        print_message(message)

# Run the analysis
await basic_stock_analysis()

EXERCISE 1A: Basic Stock Analysis with query()
Working directory: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1
Temp directory: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp

Session initialized: 5b8eb31b...

Assistant: I'll help you analyze Apple (AAPL) stock performance over the last 3 months. Let me write a Python script to fetch the data, calculate statistics, and save the results.

Tool: Write
   File: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/aapl_analysis.py

Tool Result:
File created successfully at: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/aapl_analysis.py

Assistant: Now let me execute the script:

Tool: Bash
   → Execute Apple stock analysis script
   Command: uv run python /Users/sajal/projects

### What Just Happened?

Let's break down the workflow:

1. **Agent receives task** → Understands it needs to fetch and analyze stock data
2. **Writes Python code** → Uses `Write` tool to create a script with yfinance
3. **Executes code** → Uses `Bash` tool to run the script
4. **Reads results** → Uses `Read` tool to access output files
5. **Analyzes data** → Provides financial insights based on the results

**Key Observation**: The agent is doing *finance work*, not software development. Code is just a tool to accomplish the financial analysis task.

---

### Exercise 1B: Compare Multiple Stocks

Now let's try a more complex task: comparing multiple stocks.

In [ ]:
async def compare_stocks():
    """Compare performance of multiple stocks."""
    
    print("\n" + "=" * 60)
    print("EXERCISE 1B: Multi-Stock Comparison")
    print("=" * 60)
    
    cwd = Path.cwd()
    tmp_dir = cwd / "tmp"
    tmp_dir.mkdir(exist_ok=True)
    tmp_path = str(tmp_dir.absolute())
    
    options = ClaudeAgentOptions(
        allowed_tools=["Write", "Bash", "Read"],
        permission_mode="bypassPermissions",
        model="claude-sonnet-4-5",
        max_turns=10,
        cwd=str(cwd),
        system_prompt=f"""You are a financial analysis assistant.

IMPORTANT:
- Use `uv run python` to execute Python scripts
- Save all files using ABSOLUTE PATH: {tmp_path}/filename
- DO NOT use /tmp - use {tmp_path} instead
- All packages (yfinance, pandas, etc.) are pre-installed via uv"""
    )
    
    prompt = f"""
    Compare the performance of AAPL, MSFT, and GOOGL over the last 6 months.
    
    Write Python code to:
    1. Fetch data for all three stocks
    2. Calculate:
       - Total return for each stock
       - Volatility (standard deviation of daily returns)
       - Correlation between the stocks
    3. Save results to {tmp_path}/comparison.csv
    
    Then provide an investment analysis comparing these stocks.
    """
    
    async for message in query(prompt=prompt, options=options):
        print_message(message)

# Run the comparison
await compare_stocks()


EXERCISE 1B: Multi-Stock Comparison

Session initialized: 926160a9...

Assistant: I'll help you analyze and compare the performance of AAPL, MSFT, and GOOGL over the last 6 months. Let me create a Python script to fetch the data and perform the analysis.

Tool: Write
   File: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/stock_comparison.py

Tool Result:
File created successfully at: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/stock_comparison.py

Assistant: Now let me run the script to fetch the data and perform the analysis:

Tool: Bash
   → Run stock comparison analysis script
   Command: cd /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exer...

Tool Result:
Fetching data from 2025-08-07 to 2026-02-03
------------------------------------------------------------
Fetching data for AAPL...
Fetching 

### Understanding Message Types

You saw several message types in the output:

| Message Type | Purpose | Example Content |
|--------------|---------|------------------|
| `SystemMessage` | Session initialization | Session ID, available tools |
| `AssistantMessage` | Agent's responses and tool calls | Text reasoning, tool use requests |
| `UserMessage` | Tool execution results | Output from Write, Bash, Read |
| `ResultMessage` | Final summary | Cost, duration, usage stats |

---

## Part 2: Using `ClaudeSDKClient` for Interactive Analysis

### When to Use `ClaudeSDKClient`

`ClaudeSDKClient` is better for:
- Multi-turn conversations with context
- Building interactive applications
- When users ask follow-up questions
- Complex workflows that evolve based on results

### Exercise 2A: Build an Interactive Financial Advisor

Now let's build something truly interactive: a financial advisor chatbot where **YOU** can ask follow-up questions based on the analysis.

In [18]:
from claude_agent_sdk import ClaudeSDKClient, ClaudeAgentOptions

async def interactive_financial_advisor():
    """Interactive financial advisor - ask your own questions!"""
    
    print("\n" + "=" * 60)
    print("EXERCISE 2A: Interactive Financial Advisor")
    print("=" * 60)
    print("\nThis is a real interactive session! You can ask follow-up")
    print("questions and the agent will remember the conversation context.")
    print("\nType 'done' or 'quit' to end the session.")
    print("=" * 60)
    
    cwd = Path.cwd()
    tmp_dir = cwd / "tmp"
    tmp_dir.mkdir(exist_ok=True)
    tmp_path = str(tmp_dir.absolute())
    
    print(f"\nWorking directory: {cwd}")
    print(f"Temp directory: {tmp_path}")
    
    options = ClaudeAgentOptions(
        allowed_tools=["Write", "Bash", "Read"],
        permission_mode="bypassPermissions",
        model="claude-sonnet-4-5",
        max_turns=20,
        cwd=str(cwd),
        system_prompt=f"""You are a financial analysis assistant helping users understand stock performance.

IMPORTANT:
- Use `uv run python` to execute Python scripts
- Save all files using ABSOLUTE PATH: {tmp_path}/filename
- DO NOT use /tmp - use {tmp_path} instead
- All packages (yfinance, pandas, etc.) are pre-installed via uv

Be conversational and helpful. When users ask follow-up questions, use the context from previous analyses."""
    )
    
    # Create a persistent session
    async with ClaudeSDKClient(options=options) as client:
        
        # Initial question to start the conversation
        print("\n" + "-" * 60)
        print("Starting with an initial analysis...")
        print("-" * 60)
        
        initial_query = f"""
        Analyze Netflix (NFLX) stock performance over the last year.
        Calculate key metrics like return, volatility, and save the results to {tmp_path}/nflx_analysis.json
        
        Give me a brief summary of the findings.
        """
        
        await client.query(initial_query)
        
        async for message in client.receive_response():
            print_message(message)
        
        # Now enter interactive loop
        print("\n" + "=" * 60)
        print("Initial analysis complete! Now you can ask follow-up questions.")
        print("=" * 60)
        
        turn_number = 1
        
        while True:
            print(f"\n{'='*60}")
            print(f"YOUR TURN {turn_number} - Ask a follow-up question")
            print(f"{'='*60}")
            
            # Get user input
            user_question = input("\nYour question (or 'done' to exit): ").strip()
            
            if user_question.lower() in ['done', 'quit', 'exit', '']:
                print("\n" + "=" * 60)
                print("Session ended. Thanks for using the financial advisor!")
                print("=" * 60)
                break
            
            print(f"\n{'-'*60}")
            print(f"Agent Response to Turn {turn_number}")
            print(f"{'-'*60}")
            
            # Send user's question to the agent
            await client.query(user_question)
            
            # Stream the response
            async for message in client.receive_response():
                print_message(message)
            
            turn_number += 1

# Run the interactive advisor
await interactive_financial_advisor()


EXERCISE 2A: Interactive Financial Advisor

This is a real interactive session! You can ask follow-up
questions and the agent will remember the conversation context.

Type 'done' or 'quit' to end the session.

Working directory: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1
Temp directory: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp

------------------------------------------------------------
Starting with an initial analysis...
------------------------------------------------------------

Session initialized: 8f4156dc...

Assistant: I'll analyze Netflix (NFLX) stock performance over the last year and calculate the key metrics for you.

Let me create a Python script to fetch the data, perform the analysis, and save the results.

Tool: Write
   File: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercis


Your question (or 'done' to exit):  are there other stocks for big streamers that performed better or worse?



------------------------------------------------------------
Agent Response to Turn 1
------------------------------------------------------------

Session initialized: 8f4156dc...

Assistant: Great question! Let me analyze the other major streaming companies to compare their performance against Netflix. I'll look at Disney (DIS), Warner Bros Discovery (WBD), Paramount (PARA), and maybe Roku (ROKU) as a streaming platform.

Tool: Write
   File: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/compare_streamers.py

Tool Result:
File created successfully at: /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exercises/module_1/tmp/compare_streamers.py

Tool: Bash
   → Run streaming stocks comparison analysis
   Command: cd /Users/sajal/projects/oreilly/claude_agent_sdk_course/claude_agent_sdk_course_code/notebooks/exer...

Tool Result:
Analyzing streaming stocks from 2025-02-03 t


Your question (or 'done' to exit):  exit



Session ended. Thanks for using the financial advisor!
